In [1]:
from typing import Callable
import numpy as np
import pandas as pd
import time
import torch
import torch.nn as nn
from scipy.integrate import solve_ivp

# ODE System
Here we load all the encessary physiological data for the PBPK model,
we set up the ODE system as a function, the initial conditions are defined and finally the 
equations are solved.

In [2]:
# A function to calculate the physiological parameters of the model
# given the mass of the rat and the compartments of the model

def create_params(comp_names, w):
    all_comps = ["RoB","Heart", "Kidneys", "Brain", "Spleen", "Lungs", 
                 "Liver", "Uterus", "Bone", "Adipose", "Skin", "Muscles",
                 "GIT"]

    # Density of tissues/organs
    d_tissue = 1 #g/ml
    d_skeleton = 1.92 #g/ml
    d_adipose = 0.940 #g/ml

    Q_total =(1.54*w**0.75)*60 # Total Cardiac Output (ml/h)

    Total_Blood = 0.06*w+0.77 # Total blood volume (ml)

    fr_ad = 0.0199*w + 1.644 # w in g,  Brown et al.1997 p.420. This equation gives the  adipose % of body weight 

    #read data from excel
    fractions = pd.read_excel(r'Rat physiological parameters.xlsx')

    #Tissue weight fraction 
    Tissue_fractions = fractions.iloc[:,1]/100 # % of BW. Na values refers to the volume of the rest organs(RoB)
    Tissue_fractions.index = fractions.iloc[:,0] # replace the row names with the corresponding organ 
    Tissue_fractions[9] = fr_ad/100
    #Regional blood flow fraction
    Regional_flow_fractions = fractions.iloc[:,2]/100 # % of total cardiac output
    Regional_flow_fractions.index = fractions.iloc[:,0] # replace the row names with the corresponding organ
    #Capillary volume fractions (fractions of tissue volume)
    Capillary_fractions = fractions.iloc[:,3] # of tissue volume
    Capillary_fractions.index = fractions.iloc[:,0] # replace the row names with the corresponding organ

    W_tis = np.zeros(len(comp_names))
    V_tis = np.zeros(len(comp_names))
    V_cap = np.zeros(len(comp_names))
    Q = np.zeros(len(comp_names))

    # The following values were calculated by dividing the %ID/ g tissue with the %ID w/o free 48 from Table 2 of Kreyling et al. (2017)
    # Thus, they represent the average mass, in grams, of the respective tissues in each time group.
    liver_expw = np.mean([8.57, 8.92, 9.30, 8.61, 9.20])
    spleen_expw = np.mean([0.93, 0.75, 0.97, 0.68, 0.71])
    kidneys_expw = np.mean([2.27, 2.36, 2.44, 2.11, 2.26])
    lungs_expw = np.mean([1.87, 1.60, 1.80, 1.48, 1.31])
    heart_expw = np.mean([0.89, 1.00, 1.00, 1.00, 0.88])
    blood_expw = np.mean([16.52, 17.45, 15.33, 18.50, 18.00])
    carcass_expw = np.mean([206.00, 203.33, 184.00, 202.00, 203.75])
    skeleton_expw = np.mean([26.15, 27.50, 25.56, 25.79, 25.26])
    soft_tissues = np.mean([228.57, 253.85, 214.29, 225.93, 231.04])

    # Calculation of tissue weights  
    W_tis[1] = heart_expw
    W_tis[2] = kidneys_expw
    W_tis[4] = spleen_expw
    W_tis[5] = lungs_expw
    W_tis[6] = liver_expw
    W_tis[8] = skeleton_expw
    W_tis[12] = Tissue_fractions[12]*w

    for i in range(len(comp_names)):
        control = comp_names[i]
        if control == "NA":
            Regional_flow_fractions[i] = np.nan
            Capillary_fractions.iloc[i] = np.nan
        #Calculation of tissue volumes
        if i==8:
            V_tis[i] = W_tis[i]/d_skeleton
        elif i==9:
            V_tis[i] = W_tis[i]/d_adipose
        else:
            V_tis[i] = W_tis[i]/d_tissue 

        #Calculation of capillary volumes
        V_cap[i] = V_tis[i]*Capillary_fractions[i]

        #Calculation of regional blood flows
        Q[i] = Q_total*Regional_flow_fractions[i]


    # Calculations for "Soft tissue" compartment        
    W_tis[0] = w - W_tis[1:].sum() - Total_Blood 
    V_tis[0] = W_tis[0]/d_adipose
    Q[0] = 2*Q_total - np.nansum(Q[1:])
    V_cap[0] = V_tis[0]*Capillary_fractions[0]

    V_ven=0.64*Total_Blood
    V_art=0.15*Total_Blood
    Wm_ven=0.01*V_ven
    Wm_art=0.01*V_art
    
    V_blood=Total_Blood
    
    w_rob, w_ht, w_ki, w_spl, w_lu, w_li, w_bone, w_git = W_tis[[0,1,2,4,5,6,8,12]]
    V_tis_rob, V_tis_ht, V_tis_ki, V_tis_spl, V_tis_lu, V_tis_li, V_tis_bone, V_tis_git = V_tis[[0,1,2,4,5,6,8,12]]
    V_cap_rob, V_cap_ht, V_cap_ki, V_cap_spl, V_cap_lu, V_cap_li, V_cap_bone, V_cap_git = V_cap[[0,1,2,4,5,6,8,12]]    
    Q_rob, Q_ht, Q_ki, Q_spl, Q_lu, Q_li, Q_bone, Q_git = Q[[0,1,2,4,5,6,8,12]]
    
    
    
    return[Q_total, V_blood, V_ven, V_art,
           w_rob, w_ht, w_ki, w_spl, w_lu, w_li, w_bone, w_git,
           V_tis_rob, V_tis_ht, V_tis_ki, V_tis_spl, V_tis_lu, V_tis_li, V_tis_bone, V_tis_git,
           V_cap_rob, V_cap_ht, V_cap_ki, V_cap_spl, V_cap_lu, V_cap_li, V_cap_bone, V_cap_git,
           Q_rob, Q_ht, Q_ki, Q_spl, Q_lu, Q_li, Q_bone, Q_git
          ]
    
    

In [3]:
# A function to set up the initial conditions of the ODEs.
def create_inits(dose):
    M_ht=0; M_lu=0; M_li=0; M_spl=0; 
    M_ki=0; M_git=0; M_bone=0; M_rob=0;

    M_cap_ht=0; M_cap_lu=0; 
    M_cap_li=0; M_cap_spl=0; 
    M_cap_ki=0; M_cap_git=0; 
    M_cap_bone=0; M_cap_rob=0;

    M_lumen = 0;
    M_ven = dose; M_art=0
    M_feces=0; M_urine=0 
    
    return[M_ht, M_lu, M_li, M_spl, M_ki, M_git, M_bone, M_rob,
           M_cap_ht, M_cap_lu, M_cap_li, M_cap_spl, M_cap_ki, M_cap_git, M_cap_bone, M_cap_rob,
           M_lumen, M_ven, M_art, M_feces, M_urine
          ]
    

In [4]:
# The ODEs in a function 
def ode_func(t, m, params, x):
    M_ht, M_lu, M_li, M_spl, M_ki, M_git, M_bone, M_rob, \
    M_cap_ht, M_cap_lu, M_cap_li, M_cap_spl, M_cap_ki, \
    M_cap_git, M_cap_bone, M_cap_rob, \
    M_lumen, M_ven, M_art, M_feces, M_urine = m
        
    Q_total, V_blood, V_ven, V_art, \
    w_rob, w_ht, w_ki, w_spl, w_lu, w_li, w_bone, w_git, \
    V_tis_rob, V_tis_ht, V_tis_ki, V_tis_spl, V_tis_lu, V_tis_li, V_tis_bone, V_tis_git, \
    V_cap_rob, V_cap_ht, V_cap_ki, V_cap_spl, V_cap_lu, V_cap_li, V_cap_bone, V_cap_git, \
    Q_rob, Q_ht, Q_ki, Q_spl, Q_lu, Q_li, Q_bone, Q_git = params 
    
    P_ht,P_lu,P_li,P_spl,P_ki,P_git,P_bone,P_rob, \
    x_ht,x_lu,x_li,x_spl,x_ki,x_git,x_bone,x_rob,CLE_f,CLE_h = x 
    
    CLE_u = 0
    
    # Concentrations (mg of NPs)/(g of wet tissue)
    C_ht = M_ht/w_ht
    C_cap_ht = M_cap_ht/V_cap_ht
    C_lu = M_lu/w_lu
    C_cap_lu = M_cap_lu/V_cap_lu
    C_li = M_li/w_li
    C_cap_li = M_cap_li/V_cap_li
    C_spl = M_spl/w_spl
    C_cap_spl = M_cap_spl/V_cap_spl
    C_ki = M_ki/w_ki
    C_cap_ki = M_cap_ki/V_cap_ki
    C_git = M_git/w_git
    C_cap_git = M_cap_git/V_cap_git
    C_bone = M_bone/w_bone
    C_cap_bone = M_cap_bone/V_cap_bone
    C_rob = M_rob/w_rob
    C_cap_rob = M_cap_rob/V_cap_rob
    
    C_ven = M_ven/V_ven
    C_art = M_art/V_art
    
    # Heart
    dM_cap_ht = Q_ht*(C_art - C_cap_ht) - x_ht*Q_ht*(C_cap_ht - C_ht/P_ht)
    dM_ht = x_ht*Q_ht*(C_cap_ht - C_ht/P_ht) 

    # Lungs
    dM_cap_lu = Q_total*(C_ven - C_cap_lu) - x_lu*Q_total*(C_cap_lu - C_lu/P_lu)
    dM_lu = x_lu*Q_total*(C_cap_lu - C_lu/P_lu)

    # Liver 
    dM_cap_li = Q_li*(C_art - C_cap_li) + Q_spl*(C_cap_spl - C_cap_li) + Q_git*(C_cap_git - C_cap_li) - \
                x_li*(Q_li)*(C_cap_li - C_li/P_li)
    dM_li = x_li*Q_li*(C_cap_li - C_li/P_li) - CLE_h*M_li

    # Spleen
    dM_cap_spl = Q_spl*(C_art - C_cap_spl) - x_spl*Q_spl*(C_cap_spl - C_spl/P_spl)
    dM_spl = x_spl*Q_spl*(C_cap_spl - C_spl/P_spl) 

    # Kidneys
    dM_cap_ki = Q_ki*(C_art - C_cap_ki) - x_ki*Q_ki*(C_cap_ki - C_ki/P_ki)- CLE_u*M_cap_ki
    dM_ki = x_ki*Q_ki*(C_cap_ki - C_ki/P_ki) 

    # GIT - Gastrointestinal Tract
    dM_cap_git = Q_git*(C_art - C_cap_git) - x_git*Q_git*(C_cap_git - C_git/P_git)
    dM_git = x_git*Q_git*(C_cap_git - C_git/P_git) 
    dM_lumen = CLE_h*M_li - CLE_f *M_lumen 

    # Bone
    dM_cap_bone = Q_bone*(C_art - C_cap_bone) - x_bone*Q_bone*(C_cap_bone - C_bone/P_bone)
    dM_bone = x_bone*Q_bone*(C_cap_bone - C_bone/P_bone) 


    # RoB - Rest of Body
    dM_cap_rob = Q_rob*(C_art - C_cap_rob) - x_rob*Q_rob*(C_cap_rob - C_rob/P_rob)
    dM_rob = x_rob*Q_rob*(C_cap_rob - C_rob/P_rob) 

    # Urine
    dM_urine = CLE_u*M_cap_ki

    # Feces
    dM_feces = CLE_f*M_lumen

    # Venous Blood
    dM_ven = Q_ht*C_cap_ht + (Q_li + Q_spl+Q_git)*C_cap_li + Q_ki*C_cap_ki + \
             Q_bone*C_cap_bone + Q_rob*C_cap_rob - Q_total*C_ven

    # Arterial Blood
    dM_art = Q_total*C_cap_lu - Q_total*C_art
    
    Blood_total = M_ven + M_art + M_cap_ht + M_cap_lu +M_cap_li+M_cap_spl+ \
                   M_cap_ki+ M_cap_git+M_cap_bone+M_cap_rob
    Blood = Blood_total/(V_blood)
    
    C_soft = (M_git+M_lumen+M_rob)/(w_git + w_rob)
    
    return[dM_ht, dM_lu, dM_li, dM_spl, dM_ki, dM_git, 
           dM_bone, dM_rob, dM_cap_ht, dM_cap_lu, 
           dM_cap_li, dM_cap_spl, dM_cap_ki, dM_cap_git, 
           dM_cap_bone, dM_cap_rob,
           dM_lumen, dM_ven, dM_art, 
           dM_feces, dM_urine]
#            Blood,
#            C_ht, C_lu, C_li, C_spl, 
#            C_ki, C_bone, C_soft,
#            M_feces]

In [5]:
# A function to create x values
def create_x(x):
    P1,P2,P3,P4,P5,P6,P7,P8,X1,X2,X3,X4,X5,X6,X7,X8,CLE_f,CLE_h = x
    return[P1,P2,P3,P4,P5,P6,P7,P8,X1,X2,X3,X4,X5,X6,X7,X8,CLE_f,CLE_h]

In [6]:
# Input
comp_names = ["RoB","Heart", "Kidneys", "NA", "Spleen",
              "Lungs", "Liver", "NA", "Bone","NA", "NA", "NA", "GIT"]

w = 263 # total rat mass in g
params = create_params(comp_names, w)

dose = 18.15
inits = create_inits(dose)

x_values = [1.584382e+01, 9.739224e+00, 1.524937e+01, 8.996928e+00, 1.284654e+01, 1.781151e+01,
            1.172908e+01, 1.469810e+01, 2.071897e-02, 2.097445e-02, 2.026367e-02, 4.340532e-02,
            1.857050e-02, 4.358142e-02, 2.086843e-02, 3.243307e-02, 1.546329e-01, 9.864152e-05]
x = create_x(x_values)

t_span = (0, 28*24)
#t_span = (0,9)

t_eval = range(0, 28*24+1, 1)
#t_eval = range(0,10,1)


C:\Users\vassi\AppData\Local\Temp\ipykernel_9712\1515324740.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Capillary_fractions.iloc[i] = np.nan


In [7]:
# LSODA 
start_time = time.time()
solution_LSODA = solve_ivp(fun=ode_func, t_span=t_span, method='LSODA', t_eval=t_eval, 
                     atol = 1e-05, rtol = 1e-05,
                     y0=inits, args=[params, x])
finish_time = time.time()
LSODA_time = (finish_time - start_time)
LSODA_time

0.0834357738494873

In [8]:
# BDF 
start_time = time.time()
solution_BDF = solve_ivp(fun=ode_func, t_span=t_span, method='BDF', t_eval=t_eval, 
                     atol = 1e-5, rtol = 1e-5,
                     y0=inits, args=[params, x])
finish_time = time.time()
BDF_time = (finish_time - start_time)
BDF_time

0.1119384765625

In [9]:
column_names = ['M_ht', 'M_lu', 'M_li', 'M_spl', 'M_ki', 'M_git', 'M_bone', 'M_rob',
                'M_cap_ht', 'M_cap_lu', 'M_cap_li', 'M_cap_spl', 'M_cap_ki', 'M_cap_git', 'M_cap_bone', 'M_cap_rob',
                'M_lumen', 'M_ven', 'M_art', 'M_feces', 'M_urine']

solution_df = pd.DataFrame(solution_LSODA.y.T, columns = column_names)
#solution_df

# Load Experimental Data (Kreyling: Rat IV TiO2 Data)

## Tissue biodistribution data

In [10]:
# Data are loaded as % of dose per g of tissue
data = pd.read_excel(r'Kreyling-IV-data.xlsx', sheet_name='percent_dose_per_g') 
time_points = [1,4,24,7*24,28*24] # hours
data

,Time,Liver,Spleen,Kidneys,Lungs,Heart,Brain,Uterus,Blood,Carcass,Skeleton,Soft
0,1 h,11.14,2.51,0.0230,0.063,0.009,0.0000,0.006,0.031,0.005,0.026,0.0014
1,4 h,10.62,3.43,0.0190,0.178,0.007,0.0000,0.006,0.024,0.006,0.032,0.0013
2,24 h,10.16,2.32,0.0131,0.044,0.002,0.0003,0.005,0.015,0.005,0.036,0.0007
3,7 d,10.74,4.06,0.0530,0.060,0.004,0.0000,0.006,0.002,0.005,0.019,0.0027
4,28 d,9.67,3.49,0.0760,0.041,0.008,0.0006,0.002,0.004,0.008,0.038,0.0029


In [11]:
#Drop the "Time", "Uterus", "Brain", "Carcass" columns
data.drop(["Time", "Uterus", "Brain", "Carcass"], inplace=True, axis=1)
data

,Liver,Spleen,Kidneys,Lungs,Heart,Blood,Skeleton,Soft
0,11.14,2.51,0.0230,0.063,0.009,0.031,0.026,0.0014
1,10.62,3.43,0.0190,0.178,0.007,0.024,0.032,0.0013
2,10.16,2.32,0.0131,0.044,0.002,0.015,0.036,0.0007
3,10.74,4.06,0.0530,0.060,0.004,0.002,0.019,0.0027
4,9.67,3.49,0.0760,0.041,0.008,0.004,0.038,0.0029


In [12]:
# Multiply the % percentages with the dose to calulate actual concentrations
data = data.multiply(dose/100) # Concentrations in (ug of NPs)/(g of tissue)
data

,Liver,Spleen,Kidneys,Lungs,Heart,Blood,Skeleton,Soft
0,2.021910,0.455565,0.004174,0.011435,0.001633,0.005626,0.004719,0.000254
1,1.927530,0.622545,0.003448,0.032307,0.001270,0.004356,0.005808,0.000236
2,1.844040,0.421080,0.002378,0.007986,0.000363,0.002722,0.006534,0.000127
3,1.949310,0.736890,0.009619,0.010890,0.000726,0.000363,0.003448,0.000490
4,1.755105,0.633435,0.013794,0.007442,0.001452,0.000726,0.006897,0.000526


## Fecal excretion data

In [13]:
feces_data = pd.read_excel(r'Kreyling-IV-data.xlsx', sheet_name='Feces') # load feces data
feces_time = feces_data.iloc[:,0] # Take the time points of observations for feces
feces_time = feces_time.multiply(24) # Transform from days toy hours
feces_time = feces_time.rename('Time')
feces_time

0      0.988235
1      4.000000
2     24.073946
3    167.052009
4    671.852108
Name: Time, dtype: float64

In [14]:
feces_data.drop(['Time (d)', 'SD (% ID)'], inplace=True, axis=1) # drop the Time column from the feces data
feces_data = feces_data.multiply(dose/100) # ug of 
feces_data.columns = ["Feces (ug)"]
feces_data

,Feces (ug)
0,0.023058
1,0.016336
2,0.012435
3,0.162577
4,0.484253


# Set up PINN

In [15]:
cuda_index = 0

cpu = torch.device('cpu')     # Default CUDA device
cuda = torch.device('cuda:{}'.format(cuda_index))     # Default CUDA device
device = cuda

if torch.cuda.is_available() is False:
    device = cpu


default_tensor_type = "torch.DoubleTensor"

In [16]:
class FFN(nn.Module):
    def __init__(self, input_dim, output_dim, N_hidden, hidden_dim, activation):
        super().__init__()
        
        self.input_layer = nn.Linear(input_dim, hidden_dim) #Input layer
        self.hidden_layers = nn.ModuleList(              
            [nn.Linear(hidden_dim, hidden_dim) for _ in range(N_hidden)] # Hidden layers
        )
        self.output_layer = nn.Linear(hidden_dim, output_dim) # Output layer
        self.activation = activation
        
        def forward(self, x):
            out = self.input_layer(self.input_layer(x))
            for layer in self.hidden_layers:
                out = self.activation(layer(out))
            out = self.output_layer(out)
            return out
        
#         def predict(self, x):
#             return self.forward(x)
        
# # based on /basic-pinn.logistic_equation_1d.ipynb
# class NNApproximator(nn.Module):
#     def __init__(self, dim_input: int, dim_output: int,
#                  num_hidden: int, dim_hidden: int, act=nn.Tanh()):

#         super().__init__()

#         self.layer_in = nn.Linear(dim_input, dim_hidden)
#         self.layer_out = nn.Linear(dim_hidden, dim_output)

#         num_middle = num_hidden - 1
#         self.middle_layers = nn.ModuleList(
#             [nn.Linear(dim_hidden, dim_hidden) for _ in range(num_middle)]
#         )
#         self.act = act

#     def forward(self, x):
#         out = self.act(self.layer_in(x))
#         for layer in self.middle_layers:
#             out = self.act(layer(out))
#         return self.layer_out(out)

In [17]:
def f(nn, x):
    return nn(x)

In [18]:
def df(nn, x, order=1):
    f_values = f(nn,x)
    df_values = torch.autograd.grad(outputs=f_values,
                                   inputs=x)[0]
    return df_values

In [19]:
# def f(nn: NNApproximator, x: torch.Tensor) -> torch.Tensor:
#     """Compute the value of the approximate solution from the NN model"""
#     return nn(x)
# def df(nn: NNApproximator, x: torch.Tensor = None, order: int = 1) -> torch.Tensor:
#     """Compute neural network derivative with respect to input features using PyTorch autograd engine"""
#     df_value = f(nn, x)
#     for _ in range(order):
#         df_value = torch.autograd.grad(
#             df_value,
#             x,
#             grad_outputs=torch.ones_like(x),
#             create_graph=True,
#             retain_graph=True,
#         )[0]

#     return df_value

In [20]:
# def compute_loss(
#     nn: NNApproximator, x: torch.Tensor = None, verbose: bool = False
# ) -> torch.float:
#     """Compute the full loss function as interior loss + boundary loss

#     This custom loss function is fully defined with differentiable tensors therefore
#     the .backward() method can be applied to it
#     """
#     lhs_df = df(nn, x) # the values of derivatives calculated by autograd from the NN
#     rhs_df = ode_func(x) # the values of derivatives calculated by the equations given the values of state variables from th NN
#     interior_loss = lhs_df - rhs_df
#     final_loss = interior_loss
    
# #     boundary = torch.Tensor([0.0])
# #     boundary.requires_grad = True
# #     boundary_loss = f(nn, boundary) - F0
# #     final_loss = interior_loss.pow(2).mean() + boundary_loss ** 2
#     return final_loss

In [21]:
def compute_loss(nn, x, phys_params, kinetics_params):
    lhs_df = df(nn, x) # the values of derivatives calculated by autograd from the NN
    rhs_df = ode_func(0, x, phys_params, kinetics_params) # the values of derivatives calculated by the equations given the values of state variables from th NN
    final_loss = lhs_df - rhs_df
    return final_loss

In [22]:
# Input parameters
N_input = 2 # the input is time and the dose
N_output = 21 # the number of the state variables
N_hidden = 2 # number of hidden layers
N_neurons = 10 # number of neurons in hidden layers
activation = nn.Tanh()

net = FFN(input_dim=N_input,
                      output_dim=N_output,
                      N_hidden=N_hidden,
                      hidden_dim=N_neurons,
                      activation=activation)
net

FFN(
  (input_layer): Linear(in_features=2, out_features=10, bias=True)
  (hidden_layers): ModuleList(
    (0): Linear(in_features=10, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=10, bias=True)
  )
  (output_layer): Linear(in_features=10, out_features=21, bias=True)
  (activation): Tanh()
)

In [48]:
# Produce data from the ode solution
solution_df
injections = np.zeros(solution_df.shape[0]-1)
injections[0] = dose
#x_train 
times = np.arange(0,28*24)

net_iput = np.stack((times, injections), axis=1)
net_iput = torch.from_numpy(net_iput)
type(net_iput)
print(net_iput)



tensor([[  0.0000,  18.1500],
        [  1.0000,   0.0000],
        [  2.0000,   0.0000],
        ...,
        [669.0000,   0.0000],
        [670.0000,   0.0000],
        [671.0000,   0.0000]], dtype=torch.float64)


In [44]:
# Train the model
learning_rate = 0.001
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

loss_evolution = []


In [25]:
def train_model(
    nn: NNApproximator,
    loss_fn: Callable,
    learning_rate: int = 0.01,
    max_epochs: int = 100,
) -> NNApproximator:

    loss_evolution = []

    optimizer = torch.optim.SGD(nn.parameters(), lr=learning_rate)
    for epoch in range(max_epochs):

        try:

            loss: torch.Tensor = loss_fn(nn)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if epoch % 1000 == 0:
                print(f"Epoch: {epoch} - Loss: {float(loss):>7f}")

            loss_evolution.append(loss.detach().numpy())

        except KeyboardInterrupt:
            break

    return nn, np.array(loss_evolution)

In [19]:
nn_approximator = NNApproximator(dim_input=N_input, dim_output=N_output,
                 num_hidden=N_hidden, dim_hidden=N_neurons)
nn_approximator

NNApproximator(
  (layer_in): Linear(in_features=2, out_features=10, bias=True)
  (layer_out): Linear(in_features=10, out_features=21, bias=True)
  (middle_layers): ModuleList(
    (0): Linear(in_features=10, out_features=10, bias=True)
  )
  (act): Tanh()
)